In [48]:
"""
Created on Tue Nov 15 2022
@author: Juliane Rosemeier
"""

<big> <big> <big> 1D example with fast oscillations
    
Consider
$$\dot u = iru - u^2 . $$
Apply the transformation $w = exp(irt)u ,$
which lead to the new problem
$$\dot w = - \exp(irt)w^2 ,$$
which is to be solved in this notebook.

In [32]:
"""Import modules""" 

import numpy as np
import matplotlib.pyplot as plt


In [33]:
"""Define imaginary unit"""

I = complex(0., 1.)


The exact solution of the problem is given by
$$w(t) = \frac{r w_0}{-iw_0 \exp(irt) + iw_0 +r} .$$

In [34]:
"""Compute exact solution""" 

w0 = 1.
x_start = 1.

def exact_solution(grid):
    
    N = len(grid)
    e_sol = np.zeros(N, complex)
    
    for n in range(N):
        e_sol[n] = r*w0/((-I)*w0*np.exp(I*r*grid[n])+I*w0+r)
    
    return e_sol


In [35]:
"""RHS of the problem"""

def rhs3(t,x):
    return -np.exp(I*r*t)*x*x


In [36]:
"""Runge-Kutta method of second order (RK2)"""


def RK2(f,x,t,delta_t):   
    k1 = f(t,x)
    k2 = f(t+0.5*delta_t, x + 0.5*delta_t*k1)    
    y = x + delta_t*k2    
    return y 

def RK2_on_interval2(f,x0,grid):
    num_points = len(grid)
    x_grid = np.zeros(num_points, complex)
    x_grid[0] = x0
    for n in range(num_points-1):
        tau = grid[n+1] - grid[n]
        x_grid[n+1] = RK2(f,x_grid[n],grid[n],tau)
    return x_grid 


Now, we want to apply averaging to mitigate the oscillations. The averaged equations are given by
$$\dot {\bar w} = (-1)\ \underbrace{\frac{1}{\eta_l} \int_{-\eta_l/2}^{\eta_l/2} \ \rho \left( \frac{s}{\eta_l} \right) \
\exp(irs) \ ds}_{\text{damping factor}} \ \exp(irt) \ w(t)^2$$
Thus, applying the averaging means multiplying the RHS with a damping factor.

In [37]:
"""Here the functions necessary to compute the damping factors are implemented."""

def scaled_rho(eta):
    """Scaled kernel function"""
    
    rho_0 = 0.007029858406609657
    
    dt = eta/1000.
    grid = np.arange(-0.5*eta+0.1*dt, 0.5*eta, dt)
    N = len(grid)
    sc_rho = np.zeros(N, float)
    
    for n in range(N):
        sc_rho[n] = 1./(eta*rho_0) * np.exp(1./((grid[n]/eta-0.5)*(grid[n]/eta+0.5)))
    
    return grid, sc_rho


def trapez(v, dt):
    """Quadrature rule"""
    
    N = len(v)
    integral = 0.
    
    for n in range(N-1):
        integral += 0.5*dt * (v[n]+v[n+1])
    
    return integral



def damping_factor(eta):
    """Compute damping factors"""
    
    sc_grid, sc_rho = scaled_rho(eta)
    rhs_grid = (1.) * np.exp(I*r*sc_grid)
    
    v = sc_rho*rhs_grid
    d_factor = trapez(v, eta/1000.)
    
    return d_factor


The RHS of the problem which is to be solved depends on the level. Thus, we implement level-dependent RHS. Furthermore the RK methods have the level as a parameter to make sure that the correct RHS is used.

In [38]:
"""Level-dependent RHS of the problem. We apply averaging if we are not on the finest level"""

def rhs_level(t,x,level):
    return -np.exp(I*r*t) * x*x * damping_factors[level]


In [39]:
"""Apply the RK2 method on the level-dependent RHS of the problem."""

def RK2_level(f,x,t,delta_t, level):   
    k1 = f(t,x, level)
    k2 = f(t+0.5*delta_t, x + 0.5*delta_t*k1, level)    
    y = x + delta_t*k2    
    return y 

def RK2_on_interval_level(f,x0,grid, level):
    num_points = len(grid)
    x_grid = np.zeros(num_points, complex)
    x_grid[0] = x0
    for n in range(num_points-1):
        tau = grid[n+1] - grid[n]
        x_grid[n+1] = RK2_level(f,x_grid[n],grid[n],tau, level)
    return x_grid 


In [40]:
"""Discrete l1-norm of the error"""

def error(v1,v2):
    vec_E = abs(v1-v2)/len(v1)
    E = 0.
    for n in range(len(v1)):
        E += vec_E[n]
    return E


In [41]:
"""Implementation of recursive Parareal V-cycle"""

# init_time is important for non-autonomous problems!
def parareal_Vcycle(x0, f, dt, len_interval, level, L_max, coarsening_factor, init_time):
    """This function does a Parareal V-cycle.
        x0 - intial guess 
        f - right handside of the problem 
        dt - coarse time step 
        len_interval - length of solution unterval 
        level - level where solutio shall be computes 
        L_max - coarsest level 
        coarsening_factor coarsening factor which relates the time steps on the levels 
        init_time - intial time
    """
    
    # coarse time grid
    time_grid = np.arange(0, len_interval + 0.1*dt, dt) + init_time
    len_grid = len(time_grid)
    
    # fine time step and fine time grid
    fine_dt = dt/coarsening_factor
    fine_grid = np.arange(0, dt+0.1*fine_dt, fine_dt)
    
    # array for fine solution
    fine_solution = np.zeros(len_grid, complex)
    fine_solution[0] = x0
    
    # array for for Parareal solution
    iterate = np.zeros(len_grid, complex)
    iterate[0] = x0
    
    
    # compute initial guess
    initial_guess = RK2_on_interval_level(f, x0, time_grid, level)
    
    # loop over coarse time steps
    time = init_time
    for steps in range(len_grid-1):
        
        # compute fine solution, work can be done in parallel
        if level == 1:
            f_sol = RK2_on_interval_level(f, initial_guess[steps], time+fine_grid, level-1)
            fine_solution[steps+1] = f_sol[len(f_sol)-1]
        else: 
            fine_solution[steps+1] = parareal_Vcycle(initial_guess[steps], f, fine_dt, dt, level-1, \
      
                                                     L_max, coarsening_factor, time)
        time = time+dt
    
    # loop over coarse time steps
    time = init_time
    for steps in range(len_grid-1): 
        
        # compute iterate
        old_RK_step = RK2_on_interval_level(f, initial_guess[steps], np.array([time, time+dt]), level)
        new_RK_step = RK2_on_interval_level(f, iterate[steps], np.array([time, time+dt]), level)
        iterate[steps+1] = new_RK_step[len(new_RK_step)-1] + fine_solution[steps+1] \
                            - old_RK_step[len(old_RK_step)-1]       
        time = time+dt
        
       
    # compute error and plot results on coarsest level    
    if level == L_max:
        
        #M = len(time_grid)
        e_sol = exact_solution(time_grid)
        
        print('Number of levels: ', L_max+1)
        print('error Parareal iterate: ',  error(iterate, e_sol))
        
        #plt.plot(time_grid, exact_solution(time_grid).real,'-o')
        #plt.plot(time_grid, initial_guess.real,'o')
        #plt.plot(time_grid, iterate.real,'.')
        #plt.legend(['exact solution', 'initial guess', 'iterate'])
        #plt.show()
    
    
    return iterate[len(iterate)-1]



In [42]:
"""test for r=100"""

r = 100.

# compute damping factors
d_factor1 = damping_factor(0.2)
d_factor2 = damping_factor(2.)
damping_factors = np.array([1., d_factor1, d_factor2])

parareal_Vcycle(x_start, rhs_level, 0.01, 1., 1, 1, 10, 0.);
parareal_Vcycle(x_start, rhs_level, 0.1, 1., 2, 2, 10, 0.);


Number of levels:  2
error Parareal iterate:  0.0002169750591733674
Number of levels:  3
error Parareal iterate:  0.00019811199764541986


In [44]:
"""test for r=1000"""

r = 1000.

# compute damping factors
d_factor1 = damping_factor(0.02)
d_factor2 = damping_factor(0.2)
d_factor3 = damping_factor(2.)
damping_factors = np.array([1., d_factor1, d_factor2, d_factor3])

parareal_Vcycle(x_start, rhs_level, 0.001, 1., 1, 1, 10, 0.);
parareal_Vcycle(x_start, rhs_level, 0.01, 1., 2, 2, 10, 0.);
parareal_Vcycle(x_start, rhs_level, 0.1, 1., 3, 3, 10, 0.);


Number of levels:  2
error Parareal iterate:  2.1847140062264943e-06
Number of levels:  3
error Parareal iterate:  2.106413305540892e-06
Number of levels:  4
error Parareal iterate:  2.251750942815425e-06


In [46]:
"""test for r=10000"""

r = 10000.

# compute damping factors
d_factor1 = damping_factor(0.002)
d_factor2 = damping_factor(0.02)
d_factor3 = damping_factor(0.2)
d_factor4 = damping_factor(2.)
damping_factors = np.array([1., d_factor1, d_factor2, d_factor3, d_factor4])

parareal_Vcycle(x_start, rhs_level, 0.00025, 1., 1, 1, 10, 0.);
parareal_Vcycle(x_start, rhs_level, 0.0025, 1., 2, 2, 10, 0.);
parareal_Vcycle(x_start, rhs_level, 0.025, 1., 3, 3, 10, 0.);
parareal_Vcycle(x_start, rhs_level, 0.25, 1., 4, 4, 10, 0.);



Number of levels:  2
error Parareal iterate:  3.0668862104273734e-07
Number of levels:  3
error Parareal iterate:  3.0480547757705495e-07
Number of levels:  4
error Parareal iterate:  3.0357016877934065e-07
Number of levels:  5
error Parareal iterate:  2.7011063136189545e-07
